# Visualize Decoded Results

In [2]:
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
import numpy as np
import shutil
from PIL import Image
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from sphere import Sphere
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from functools import partial
from dh_grid import DHGrid
from sphere import Sphere
from img_sphere import ImageSphere
from metrics import *
from average_meter import AverageMeter
import time
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/tmp/rslidar/decoded/'
# export_ds = '/media/scratch/berlukas/nuscenes'

decoded_filename = f"{export_ds}/sem_clouds_decoded.npy"
cloud_filename = f"{export_ds}/sem_clouds.npy"
decoded_clouds = np.load(decoded_filename)
cloud_features = np.load(cloud_filename)

decoded_gt = cloud_features[:, 2, :, :]
cloud_features = cloud_features[:, 0:2, :, :]

print(f"Shape of the decoded signal: {decoded_clouds.shape}.")
print(f"Shape of the input signal GT: {decoded_gt.shape}.")
print(f"Shape of the input signal: {cloud_features.shape}.")

IndexError: index 2 is out of bounds for axis 1 with size 2

In [20]:
export_ds = '/tmp/rslidar/decoded/'
# export_ds = '/media/berlukas/Data/data/nuscenes/'
cloud_filename = f"{export_ds}/sem_clouds_decoded.npy"
gt_filename = f"{export_ds}/sem_clouds.npy"
decoded_clouds = np.load(cloud_filename)
decoded_gt = np.load(gt_filename)

print(f'shape of loaded clouds is {decoded_clouds.shape} and gt is {decoded_gt.shape}')

shape of loaded clouds is (120, 6, 100, 100) and gt is (120, 2, 100, 100)


In [ ]:
export_ds = ''

dec_input_clouds = f"{export_ds}/decoded_fused_input_clouds.npy"
dec_input_images = f"{export_ds}/decoded_fused_input_images.npy"
dec_clouds = f"{export_ds}/decoded_fused.npy"
dec_gt = f"{export_ds}/decoded_fused_gt.npy"

input_clouds = np.load(dec_input_clouds)
input_images = np.load(dec_input_images)
decoded_clouds = np.load(dec_clouds)
decoded_gt = np.load(dec_gt)

In [4]:
def CreateGrid(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([2, n_grid, n_grid])
    for i in range(n_grid):
        for j in range(n_grid):
            points[0, i, j] = (np.pi*(2*i+1))/(4*bw)
            points[1, i, j] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points

def createGrid_old(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])    
    return cart_grid

def mapIntensityToRGB(i):
    # return cm.jet(plt.Normalize(min(i), max(i))(i))
    map = np.array([[0,0,0],
                    [1,0,0],
                    [0,1,0],
                    [0,0,1],
                    [1,1,0],
                    [0,1,1]])
    return map[i,:]

class SamplingPointCloud(PointCloud):

    @staticmethod
    def nbr_dims() -> int:
        return 4
    
    @classmethod
    def from_file(cls, file_name: str) -> 'SamplingPointCloud':
        return None
    
    @classmethod
    def from_bw(cls, bw, scale = 100) -> 'SamplingPointCloud':
        grid = createGrid_old(bw)
        xyz_grid = convertGridToEuclidean_old(grid) * scale
        intensities = np.zeros((xyz_grid.shape[0],1))
        sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
        return cls(sampling_grid.T)

def create_spherical_pc(feature, trans = 0, bw = 100):
    pc = SamplingPointCloud.from_bw(bw, 1)   
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = mapIntensityToRGB(feature[:, 0])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd    

def create_cloud_pc(cloud, trans = 0):
    cloud[:,0] = cloud[:,0] + trans
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd

def compare_estimation_sphere(decoded, gt, bw = 100):  
    decoded_pc = create_spherical_pc(decoded, trans=0, bw=bw)
    gt_pc = create_spherical_pc(gt, trans=2.5, bw=bw)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])    
    
def compare_estimation_clouds(decoded, gt, bw = 100):  
    decoded_pc = create_cloud_pc(decoded, trans=0)
    gt_pc = create_cloud_pc(gt, trans=100)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])
    
def backproject_cloud(spherical, distance, bw = 100):    
    grid, _ = DHGrid.CreateGrid(bw)
    n_points = grid.shape[1] * grid.shape[2]
    cart_sphere = np.zeros([n_points, 4])
    k = 0
    for i in range(0, grid.shape[1]):
        for j in range(0, grid.shape[2]):
            dist = distance[i,j]
            if dist <= 0:
                continue
            cart_sphere[k,0] = dist * np.multiply(np.cos(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,1] = dist * np.multiply(np.sin(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,2] = dist * np.cos(grid[0,i,j])
            cart_sphere[k,3] = spherical[i,j]
            k = k + 1
    return cart_sphere  

def prepare_for_viz(cloud, colors = None):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)    
    return pcd

def convert_sphere(sph, feature_idx, n_features, bw, trans = 0.0):
    sph = np.reshape(sph, (n_features, -1)).T
    
    pc = SamplingPointCloud.from_bw(bw, 1)
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    return np.column_stack((points_xyz, sph[:,feature_idx]))
    
def visualize_pointcloud(cloud, colors = None):
    pcd = prepare_for_viz(cloud, colors)
    o3d.visualization.draw_geometries([pcd], width=640,  height=480)

def visualize_sphere(sph, feature_idx = 0, n_features = 3, bw = 100):
    points_xyzi = convert_sphere(sph, feature_idx, n_features, bw)
    visualize_pointcloud(points_xyzi)
      
def compare_estimation_sphere2(decoded, gt, bw = 100):  
    decoded_pc = convert_sphere(decoded, 0, 1, 100)
    gt_pc = convert_sphere(gt, 2, 3, 100, 2.5)
    
    decoded_pcd = prepare_for_viz(decoded_pc)
    gt_pcd = prepare_for_viz(gt_pc)
    o3d.visualization.draw_geometries([decoded_pcd, gt_pcd]) 
    
def compare_estimation_sphere3(decoded, gt, feature_idx = 0, n_features = 3, bw = 100):
    decoded_pc = convert_sphere(decoded, feature_idx, n_features, bw)
    gt_pc = convert_sphere(gt, feature_idx, n_features, bw, 2.5)
    
    decoded_pcd = prepare_for_viz(decoded_pc)
    gt_pcd = prepare_for_viz(gt_pc)
    o3d.visualization.draw_geometries([decoded_pcd, gt_pcd]) 

In [10]:
n_decoded = decoded_clouds.shape[0]

avg_pixel_acc = AverageMeter()
avg_pixel_acc_per_class = AverageMeter()
avg_jacc = AverageMeter()
avg_dice = AverageMeter()
for i in range(150, 160):
    cur_decoded = decoded_clouds[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)
    cur_sem_cloud = decoded_gt[i, :, :]
    cur_input = cloud_features[i, :, :]
    
    mask = cur_sem_cloud <= 0
    cur_decoded[mask] = 0
    
    pred_segmentation = torch.from_numpy(cur_decoded).cuda().int()
    gt_segmentation = torch.from_numpy(cur_sem_cloud).cuda().int()
    pixel_acc, pixel_acc_per_class, jacc, dice = eval_metrics(gt_segmentation, pred_segmentation, num_classes = 7)
    avg_pixel_acc.update(pixel_acc)
    avg_pixel_acc_per_class.update(pixel_acc_per_class)
    avg_jacc.update(jacc)
    avg_dice.update(dice)
        
    cur_decoded = np.reshape(cur_decoded, (1, -1)).T    
    cur_sem_cloud = np.reshape(cur_sem_cloud, (1, -1)).T        
    compare_estimation_sphere(cur_decoded, cur_sem_cloud, 100)

print(f'overall pixel acc = {avg_pixel_acc.avg}')
print(f'pixel acc per class = {avg_pixel_acc_per_class.avg}')
print(f'mean jaccard index = {avg_jacc.avg}')
print(f'mean dice coeff = {avg_dice.avg}')

overall pixel acc = 0.9346949458122253
pixel acc per class = 0.4844251573085785
mean jaccard index = 0.3875047564506531
mean dice coeff = 0.466064989566803


In [23]:
# Visualize decoded pointcloud
i = 77
bw = 100

cur_decoded = decoded_clouds[i, :, :]
cur_gt = decoded_gt[i, :, :]
# visualize_sphere(cur_decoded, 0, 1, 100)
compare_estimation_sphere3(cur_decoded, cur_gt, 0, 1, 100)

# mask = cur_sem_cloud <= 0
# cur_decoded[mask] = 0

# compare_estimation_sphere2(cur_decoded, cur_cloud)

# compare_estimation_sphere(cur_decoded, cur_cloud)
# visualize_sphere(cur_cloud, 2, 3, 100)
# est_cloud = backproject_cloud(cur_decoded, cur_input[0,:,:], bw)
# gt_cloud = backproject_cloud(cur_sem_sphere, cur_input[0,:,:], bw)

In [18]:
runtime = 0
n = 400
for i in range(0,n):
    cur_decoded = decoded_clouds[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)    
    cur_sem_cloud = decoded_gt[i, :, :]
    mask = cur_sem_cloud <= 0
    cur_decoded[mask] = 0
    start_time = time.time() * 1000
    est_cloud = backproject_cloud(cur_decoded, cur_input[0,:,:], bw)
    execution_time = (time.time() * 1000 - start_time)
    runtime = runtime + execution_time
print(f'projection took {runtime / n} ms')

projection took 148.74991577148438 ms


In [56]:
# Visualize ground truth
i = 1
cur_gt = decoded_gt[i, :, :]
visualize_sphere(cur_gt, 0, 1, 100)

In [96]:
# New IoU computation
from iou import IoU

num_classes = 7
ignore_index = 0
bw=100
metric = IoU(num_classes, ignore_index=ignore_index)

for i in range(0,300):
    cur_decoded = decoded_clouds[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)    
    cur_gt = decoded_gt[i, :, :]

    mask = cur_gt <= 0   
    cur_decoded[mask] = 0
    cur_gt[mask] = 0

    pred_segmentation = torch.from_numpy(np.reshape(cur_decoded, [1, 2*bw, 2*bw])).cuda().int()
    gt_segmentation = torch.from_numpy(np.reshape(cur_gt, [1, 2*bw, 2*bw])).cuda().int()
    metric.add(pred_segmentation, gt_segmentation)
iou, miou = metric.value()

print('========================')
print(f'Mean IoU is: {miou}')
print(f'Class-wise IoU is: {iou[1:]}')    
compare_estimation_sphere(np.reshape(cur_decoded, (1, -1)).T , np.reshape(cur_gt, (1, -1)).T, 100)
# visualize_sphere(cur_decoded, 0, 1, 100)

Mean IoU is: 0.42056361505650336
Class-wise IoU is: [0.58380722 0.15450794 0.         0.89509876 0.39184103 0.49812674]


# New Viz

In [ ]:
dataroot = '/tmp/rslidar/decoded/'

clouds = sorted(os.listdir(dataroot))
n_clouds = len(clouds)
for i in tqdm(range(0, n_clouds)):
    filename = f'{dataroot}/{clouds[i]}'
    print(f'visualizing {filename}')
    cloud = np.load(filename)
    visualize_sphere(cloud, 0, 1, 50)

In [ ]:
dataroot = '/tmp/rslidar/decoded/'

sequences = sorted(os.listdir(dataroot))
n_sequences = len(clouds)
for i in tqdm(range(0, n_sequences)):
    filename = f'{dataroot}/{sequences[i]}'
    print(f'visualizing {filename}')    
    clouds = np.load(filename)
    n_clouds = clouds.shape[0]
    for j in range(0, n_clouds):
        cloud = clouds[j, :, :, :]
        cloud = np.argmax(cloud, axis=0)
        visualize_sphere(cloud, 0, 1, 50)

In [29]:
dataroot = '/media/berlukas/Data/data/datasets/s2ae/teaser/waymo/'

sequences = sorted(os.listdir(dataroot))
n_sequences = len(sequences)
for i in tqdm(range(0, n_sequences)):
    filename = f'{dataroot}/{sequences[i]}'
    print(f'Visualizing {filename}')
    
    clouds = np.load(filename)
    visualize_sphere(clouds[1], 0, 2, 50)

  0%|          | 0/1 [00:00<?, ?it/s]

Visualizing /media/berlukas/Data/data/datasets/s2ae/teaser/waymo//sem_clouds_waymo_02_val_bw50.npy


## New New Viz

In [17]:
def mapIntensityToRGB(i):
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    map = [[0,0,0, 1], # black, unlabeled
           [1,0,0, 1], # red, vehicle
           [0,0,1, 1], # green, person
           [0,0,1, 1], # blue, ground
           [1,1,0, 1], # yellow, man-made
           [0,1,1, 1]] # cyan, vegetation        
    return np.array([map[int(idx)] for idx in i])        

export_ds = '/tmp/rslidar/decoded/'
# export_ds = '/media/berlukas/Data/data/nuscenes/'
cloud_filename = f"{export_ds}/sem_clouds_decoded.npy"
input_filename = f"{export_ds}/sem_cloud_custom_val.npy"
decoded_clouds = np.load(cloud_filename)
decoded_gt = np.load(input_filename)

print(f'shape of loaded clouds is {decoded_clouds.shape} and gt is {decoded_gt.shape}')

clouds = np.load(cloud_filename)
inputs = np.load(input_filename)
n_clouds = clouds.shape[0]
n_inputs = inputs.shape[0]
assert n_clouds == n_inputs

bw = 50
for j in tqdm(range(0, n_clouds, 2)):
    cur_decoded = clouds[j, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)
    cur_input = inputs[j, :, :, :]
        
    # visualize_sphere(cur_decoded, 0, 1, 50)
    est_cloud = backproject_cloud(cur_decoded, cur_input[0,:,:], bw)
    visualize_pointcloud(est_cloud)

shape of loaded clouds is (278, 6, 100, 100) and gt is (278, 2, 100, 100)


  0%|          | 0/139 [00:00<?, ?it/s]

KeyboardInterrupt: 